In [1]:
#Machine learning 

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv(r'C:\Users\Giacomo\Desktop\DataScience\Python\DataSet\melb_data.csv')

In [4]:
data = data.set_index('Unnamed: 0');

In [5]:
data.head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
5,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [6]:
data.info()
#nan are not admitted

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18396 entries, 1 to 23546
Data columns (total 21 columns):
Suburb           18396 non-null object
Address          18396 non-null object
Rooms            18396 non-null int64
Type             18396 non-null object
Price            18396 non-null float64
Method           18396 non-null object
SellerG          18396 non-null object
Date             18396 non-null object
Distance         18395 non-null float64
Postcode         18395 non-null float64
Bedroom2         14927 non-null float64
Bathroom         14925 non-null float64
Car              14820 non-null float64
Landsize         13603 non-null float64
BuildingArea     7762 non-null float64
YearBuilt        8958 non-null float64
CouncilArea      12233 non-null object
Lattitude        15064 non-null float64
Longtitude       15064 non-null float64
Regionname       18395 non-null object
Propertycount    18395 non-null float64
dtypes: float64(12), int64(1), object(8)
memory usage: 3.1+ MB


In [7]:
# Shape of dataset (num_rows, num_columns)
print(data.shape)
# Number of missing values in each column of dataset
missing_val_count_by_column = (data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(18396, 21)
Distance             1
Postcode             1
Bedroom2          3469
Bathroom          3471
Car               3576
Landsize          4793
BuildingArea     10634
YearBuilt         9438
CouncilArea       6163
Lattitude         3332
Longtitude        3332
Regionname           1
Propertycount        1
dtype: int64


In [8]:
#first machine learning
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
#this model doesn't like NaN
data_M = data[pd.notnull(data.Bathroom)]
data_M = data_M[pd.notnull(data_M.Landsize)]
data_M = data_M[pd.notnull(data_M.Bedroom2)]
y = data_M.Price
melbourne_features = ['Rooms','Bathroom','Landsize','Bedroom2']
X = data_M[melbourne_features]
X.describe();
#Define: What type of model will it be? A decision tree? Some other type of model? Some other parameters of the model type are specified too.
#Fit: Capture patterns from provided data. This is the heart of modeling.
#Predict: Just what it sounds like
#Evaluate: Determine how accurate the model's predictions are.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
melbourne_model = DecisionTreeRegressor()
melbourne_model.fit(train_X, train_y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [9]:
val_predictions = melbourne_model.predict(val_X)
print('MAE = ',mean_absolute_error(val_y, val_predictions))
print("Making predictions for the following 5 houses:")
print(X.head())
print("The predictions are")
print(melbourne_model.predict(X.head()))
a = [int(i) for i in data_M.loc[1:6,['Price']].values]
print('The real values are', a, sep = '\n')

MAE =  453690.26740565704
Making predictions for the following 5 houses:
            Rooms  Bathroom  Landsize  Bedroom2
Unnamed: 0                                     
1               2       1.0     202.0       2.0
2               2       1.0     156.0       2.0
4               3       2.0     134.0       3.0
5               3       2.0      94.0       3.0
6               4       1.0     120.0       3.0
The predictions are
[ 995250.          964928.57142857 1062500.          850000.
 1600000.        ]
The real values are
[1480000, 1035000, 1465000, 850000, 1600000]


In [10]:
#tuning
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [11]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  397304
Max leaf nodes: 50  		 Mean Absolute Error:  373724
Max leaf nodes: 500  		 Mean Absolute Error:  398173
Max leaf nodes: 5000  		 Mean Absolute Error:  454019


In [12]:
#random forest
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor(n_estimators = 1000,max_depth = 8,random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds)/y.mean())

0.3444005763491392


In [13]:
#Missing Values

In [14]:
melbourne_features = ['YearBuilt','BuildingArea','Car','Rooms','Bathroom','Landsize','Bedroom2','Longtitude','Lattitude','Distance']
X = data[melbourne_features]
y = data['Price']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 0)

In [15]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]#if there is a nan it returns True

In [16]:
data['Rooms'].isnull().any()#in Rooms there isn't a NaN

False

In [17]:
#define a function to compare MAE from different startng points
def score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid):
    model = RandomForestRegressor(random_state=1)
    model.fit(reduced_X_train, y_train)
    preds_val = model.predict(reduced_X_valid)
    mae = mean_absolute_error(y_valid, preds_val)
    return(mae)
    

In [18]:
# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)
#Score from Approach 1 (Drop Columns with Missing Values)
print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
389110.68051817594


C:\Users\Giacomo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [19]:
#Score from Approach 2 (Imputation)
#we use SimpleImputer to replace missing values with the mean value along each column.
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):


C:\Users\Giacomo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


211332.05348453438


In [20]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):


C:\Users\Giacomo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


210718.48154044786


In [21]:
#categorical variable

In [22]:
X = data.drop(['Price'], axis = 1)
#data

In [23]:
def fill(data):
    df1 = pd.DataFrame(my_imputer.fit_transform((data.select_dtypes(exclude=['object']))))
    df1.columns = data.select_dtypes(exclude=['object']).columns
    df2 = data.select_dtypes(include=['object'])
    df = df1.join(df2)
    return df

In [24]:
data_clean = fill(data)
data_clean = data_clean[pd.notnull(data_clean['CouncilArea'])]
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11290 entries, 1 to 18395
Data columns (total 21 columns):
Rooms            11290 non-null float64
Price            11290 non-null float64
Distance         11290 non-null float64
Postcode         11290 non-null float64
Bedroom2         11290 non-null float64
Bathroom         11290 non-null float64
Car              11290 non-null float64
Landsize         11290 non-null float64
BuildingArea     11290 non-null float64
YearBuilt        11290 non-null float64
Lattitude        11290 non-null float64
Longtitude       11290 non-null float64
Propertycount    11290 non-null float64
Suburb           11290 non-null object
Address          11290 non-null object
Type             11290 non-null object
Method           11290 non-null object
SellerG          11290 non-null object
Date             11290 non-null object
CouncilArea      11290 non-null object
Regionname       11290 non-null object
dtypes: float64(13), object(8)
memory usage: 1.9+ MB


In [25]:
y = data_clean['Price']
X = data_clean.drop(['Price'], axis = 1)

In [26]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 0)

In [27]:
# Get list of categorical variables
s = (X.dtypes == 'object')
print((s))
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)

Rooms            False
Distance         False
Postcode         False
Bedroom2         False
Bathroom         False
Car              False
Landsize         False
BuildingArea     False
YearBuilt        False
Lattitude        False
Longtitude       False
Propertycount    False
Suburb            True
Address           True
Type              True
Method            True
SellerG           True
Date              True
CouncilArea       True
Regionname        True
dtype: bool
Categorical variables:
['Suburb', 'Address', 'Type', 'Method', 'SellerG', 'Date', 'CouncilArea', 'Regionname']


In [28]:
y_valid

6934     1350000.0
9013      912500.0
7768      670000.0
6107     2460000.0
17969     603000.0
           ...    
17461    2123500.0
3829      542500.0
14728     680000.0
6162      735000.0
8949     2150000.0
Name: Price, Length: 2823, dtype: float64

In [29]:
#Score from Approach 1 (Drop Categorical Variables)
drop_X_train = X_train.select_dtypes(exclude=['object'])#drop categorical variables
drop_X_valid = X_valid.select_dtypes(exclude=['object'])
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):


C:\Users\Giacomo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


203118.33872818082


In [30]:
#Score from Approach 2 (Label Encoding)
from sklearn.preprocessing import LabelEncoder
# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()
# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.fit_transform(X_valid[col])
print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Label Encoding):


C:\Users\Giacomo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


209260.27198016294


In [31]:
#Score from Approach 3 (One-Hot Encoding)
#We set handle_unknown='ignore' to avoid errors when the validation data contains classes that aren't represented in the training data, and
#setting sparse=False ensures that the encoded columns are returned as a numpy array (instead of a sparse matrix).

In [32]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))#just the object columns
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):


C:\Users\Giacomo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


202977.07173219978


In [33]:
#Pipeline

In [34]:
#Cleaner Code: Accounting for data at each step of preprocessing can get messy. With a pipeline, you won't need to manually keep track of your training and validation data at each step.
#Fewer Bugs: There are fewer opportunities to misapply a step or forget a preprocessing step.
#Easier to Productionize: It can be surprisingly hard to transition a model from a prototype to something deployable at scale. We won't go into the many related concerns here, but pipelines can help.
#More Options for Model Validation: You will see an example in the next tutorial, which covers cross-validation.

In [35]:
X = data.drop(['Price'], axis = 1)
y = data['Price']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 0)


In [36]:
X

,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
1,Abbotsford,85 Turner St,2,h,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
5,Abbotsford,40 Federation La,3,h,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
6,Abbotsford,55a Park St,4,h,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23540,Williamstown,8/2 Thompson St,2,t,SP,Greg,26/08/2017,6.8,3016.0,2.0,2.0,1.0,NaN,89.0,2010.0,NaN,-37.86393,144.90484,Western Metropolitan,6380.0
23541,Williamstown,96 Verdon St,4,h,PI,Sweeney,26/08/2017,6.8,3016.0,4.0,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0
23544,Yallambie,17 Amaroo Wy,4,h,S,Buckingham,26/08/2017,12.7,3085.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN,-37.72006,145.10547,Northern Metropolitan,1369.0


In [37]:
#We construct the full pipeline in three steps.

In [38]:
#Step 1: Define Preprocessing Steps
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore',sparse=False))
])

a = (X.dtypes == 'object')
categorical_cols = list(a[a].index)

a = (X.dtypes != 'object')
numerical_cols = list(a[a].index)

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [39]:
#Step 2: Define the Mode

In [40]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=1, random_state=0)

In [41]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 238228.65035877365


In [42]:
#cross validation

In [43]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

MAE scores:
 [290485.67907609 275769.65833107 248239.9268823  222593.27860832
 235293.71106279]


In [44]:
#XGBoost

In [ ]:
from xgboost import XGBRegressor
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model',XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=1,early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False))
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

C:\Users\Giacomo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[22:00:20] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
